<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/korean_real_summary_v1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

In [1]:
!pip install sentence-transformers==0.3.0
!pip install transformers==3.0.2
!pip install wikipedia
!pip install konlpy

     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 1.5MB 12.9MB/s 
     |████████████████████████████████| 2.9MB 43.2MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-cp36-none-any.whl size=86754 sha256=bdfc43d8691bf50a903f4f18ecf75ac438ee97fbdf384970cef1fe08693ab546
  Stored in directory: /root/.cache/pip/wheels/b8/23/85/85d6a9a6c68f0625a1ecdaad903bb0a78df058c10cf74f9de4
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4ce631f26286e89f480b3e45b86779250bf8e3cbd81c149e1e3d03fa8e42d845
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 778kB 13.4MB/s 
     |████████████████████████████████| 3.0MB 50.3MB/s 
     |████████████████████████████████| 1.1MB 48.0MB/s 
  

In [2]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [3]:
org_text = """
주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

한국어 문법체계에 따라 요약문을 생성하기 위해 한국어 문장 샘플을 준비

'한글 위키백과'에서 임의의 문장을 수집 함

In [4]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [5]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
          print(ex)
    return contents


ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

['동화', '신 전래동화', '꾸러기 수비대', '아동 문학', '호시조라 미유키', '한국의 사찰', '해와 달이 된 오누이', '잠자는 숲속의 미녀', '거북', '이한갈', '정은찬', '옛날 옛적에 (애니메이션)', '김기두 (배우)', '계룡선녀전 (드라마)', '동요', '이상훈 (1976년)', '최홍일', '육진수 (격투기 선수)', '제비', '장석현 (연예인)', '유다미', '밀교 (불교)', '최지웅', '박재훈 (배우)', '재희', '한다은', '정미남', '안젤리나 다닐로바', '도깨비', '기탄교육', '국지용', '의왕백운호수축제', '박민경', '정정아', '콩딱쿵! 이야기 주머니', '윤기원 (배우)', '선녀와 나무꾼', '남생이', '콩쥐팥쥐 (동음이의)', '대장화홍련전', '토끼전', '지대한', '이설구', '빨간 자전거', '미녀와 야수', '이서휴게소', '은비까비의 옛날옛적에', '콩쥐팥쥐전', '김덕현 (배우)', '아시리아인', '티베트', '백조의 호수', '홍석연', '서예', '도교', '켈트 다신교', '금도끼 은도끼', '제네시스 (밴드)', '장화, 홍련 (동음이의)', '허구 국가', '노상현', '타이완의 문화', '도깨비 (동음이의)', '골디락스', '자와어', '선녀강림', '손춘익', '안동국제탈춤페스티벌', '윌리엄 버틀러 예이츠', '성덕대왕신종', '모모타로', '고려', '한국 문학', '라푼젤 (영화)', '토비트', '조선 후기의 문학', '송월동 동화마을', '계룡선녀전', '홍석천', '이집트', '응우옌 왕조', '아시아의 역사', '안지환', '외래어', '프랑스인', '스키타이족', '메이플 월드', '네버랜드', '인도네시아', '프랑크인', '이탈리아', '일본 제국', 'MBC 창작동요제', '드랑 나흐 오스텐', '김환영 (작가)', '바이킹', '조선', '원나라', '오윤 (화가)', '앱북']
"콩쥐팥쥐 (동음이의)" may refer to: 
콩쥐팥쥐


In [6]:
ko_grammar_set_raw += __search_from_wiki("역사", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

['역사', '한국의 역사', '불교의 역사', '대체 역사', '일본의 역사', '스위스 역사 사전', '유럽의 역사', '고려의 역사', '세계의 역사', '조선의 역사', '동북아역사재단', '우즈베키스탄의 역사', '중국의 역사', '대한민국의 역사 드라마 목록', '한국 불교의 역사', '역사주의', '서울역사박물관', '오키나와현의 역사', '역사학', '러시아의 역사', '역사적 예수', '다큐멘터리 역사를 찾아서', '소련의 역사', '조선민주주의인민공화국의 역사', '조지아의 역사', '영국의 역사', '역사철학', '프랑스의 역사', '역사신학', '애플의 역사', '인터넷의 역사', '유엔의 역사', '역사지진', '역사저널 그날', '힌두교의 역사', '역사 시대', '역사소설', '독일의 역사', '압하지야의 역사', '아일랜드의 역사', '구글의 역사', '영화의 역사', '기독교의 역사', '운영 체제의 역사', '몬테네그로의 역사', '태국의 역사', '음소문자의 역사', '사기 (역사서)', '파키스탄의 역사', '사학자', '그리스의 역사', '베트남의 역사', '기술의 역사', '역사스페셜', '의사 역사학', '안드로이드 버전 역사', '서울역', '미국 육군 역사관', '인도의 역사', '역사언어학', '벨라루스의 역사', '퀴디치의 역사', '사극', '역사적 유물론', '마리아론의 역사', 'FC 서울의 역사', '역사 (헤로도토스)', '폭력의 역사', '교육의 역사', '중화민국의 역사', '터키의 역사', '몽골의 역사', '근대 그리스의 역사', '베트남 역사박물관', '스페인의 역사', '우크라이나의 역사', '핀란드의 역사', '맨체스터 유나이티드 FC의 역사', '음악의 역사', '이란의 역사', '에스페란토의 역사', '타이완의 역사', '민자역사', '역사 재현', '크림반도의 역사', '라오스의 역사', '제2차 세계 대전 기간 미국의 군사 역사', '브리튼 제도의 역사', '이집트의 역사', '한국사

In [7]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[120])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
한반도는 제2차 세계 대전 이후 미국과 소련의 군정기를 거쳐 조선민주주의인민공화국(북한)과 대한민국(남한)으로 나뉘게 되었다. 1948년 이전의 한반도의 역사에 관해서는 한국의 역사를 참조하도록 한다. 이 문서는 조선민주주의인민공화국의 역사에 관한 내용을 다루고 있다. 형성기 1945년 8월 15일 광복 이후, 38도선 이북에서는 소련군이 1948년 9월 9일 조선민주주의인민공화국 수립 이전까지, 38선 이남에서는 미군이 1945년 9월 8일부터 1948년 8월 15일 대한민국(남한) 정부수립까지 한반도를 다스린 기간을 군정기라 부른다. 이 기간 동안 남한에서는 수많은 정치 단체들이 조직되어 이합집산하였고, 모스크바 3상회의로 신탁통치안을 놓고 찬반으로 나뉘어 좌우익이 심각하게 대립하였다. 해방 직후 38선 이북에서는 민족주의 계열의 조만식을 위원장으로 하는 평남 건국준비위원회가 결성되었으나, 소련의 김일성 지지와 지원으로 평남건준위는 유명무실화되었고, 조만식을 비롯한 민족주의계 인사들은 김일성에 의해 숙청되었다. 1946년 2월 8일 김일성을 위원장으로 하는 북조선 임시 인민 위원회가 구성되어 사실상 정부의 구실을 하였다. 그리고서는 무상분배 토지개혁 단행, 국유화 등 개혁을 했고, 이 후 공산주의 지배 체제를 확고히 한 뒤에는 수상을 김일성으로 하고 부수상을 박헌영으로하는  1948년 9월 9일 조선민주주의인민공화국이 수립되었다. 초기에 대체로 정치 구성원은 갑산파의 김일성, 남로당파 박헌영, 연안파 김두봉, 소련파 등으로 이뤄져 연립내각체제에 있었다. 곧, 부수상 겸 외무상에 박헌영, 부수상에 홍명희, 부수상 겸 산업상에 박헌영, 국가기획위원장에 정준택, 민족보위상에 최용건, 국가검열상(국방상)에 김원봉, 내상(內相)에 박일우, 농림상에 박문규, 상상(商相)에 장지

문장으로 잘라 낸다

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 13542


In [10]:
ko_grammar_sentences[11000]

'1970년대 무지부르는 파키스탄 정부에 의해 체포되고 AL당은 활동이 금지되었다.'

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [11]:
from konlpy.tag import Twitter
twitter = Twitter()

print(twitter.pos(ko_grammar_sentences[305]))

[('백운호수', 'Noun'), ('에서', 'Josa'), ('의왕시', 'Noun'), ('와', 'Josa'), ('의왕시', 'Noun'), ('축제', 'Noun'), ('추진', 'Noun'), ('위원회', 'Noun'), ('가', 'Josa'), ('공동', 'Noun'), ('으로', 'Josa'), ('주최', 'Noun'), ('하여', 'Verb'), ('매년', 'Noun'), ('9월', 'Number'), ('에', 'Foreign'), ('이틀', 'Noun'), ('동안', 'Noun'), ('열리는', 'Verb'), ('축제', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation')]


In [12]:
# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:1000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1

In [13]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Conjunction       13
 Determiner        14
 VerbPrefix        15
 Exclamation       16
 KoreanParticle    17
 Eomi              18
 ScreenName        19
 URL               20
----------------------------------------------------------


In [14]:
# morpheme 코드 변환기
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
print(f'Code length : {len(code)}')

Code length : 67


In [15]:
# 전체 형태소 코드로 변환
ko_grammar_set = []
for sentence in ko_grammar_sentences:
    code = morpheme_encode(sentence)
    if len(code) <= _MAX_MORP_LENGTH:
        ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(13343, 128)

# Dataset 전체 구성

In [16]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [00:56<00:00, 31.9MB/s]


In [17]:
BATCH_SIZE = 64
BATCH_COUNT = 100

In [18]:
#dataset 다시 만듦

org_text_emb = embedder.encode([org_text])[0]
print(f'Text embedding shape : {org_text_emb.shape}')
dataset = []
for i in range(BATCH_COUNT):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(org_text_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))

print(f'Total dataset count :{BATCH_COUNT*BATCH_SIZE}')

Text embedding shape : (1024,)
Total dataset count :6400


In [19]:
# 30번째 배치의 형태소코드셋 중 20번째꺼 확인
print(dataset[30][1][20])
print(dataset[31][1][20])

[2 9 2 5 2 2 5 6 2 2 5 6 2 2 5 2 5 6 3 2 5 2 5 2 6 2 5 6 2 5 6 6 2 8 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 2  2  5  2  5  2  5  2  2  2  5  2  8  3  2  5  2  6  6  2  5  2  5  3
 11  3 11  3 11  3 11  3 11  3 11  3 11  3  2  2  5  2  5  6  2  6  2  5
  6  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


# Generator 구현

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([org_text])
total_words = len(tokenizer.word_index) + 1

print(f'Total token count of origin text : {total_words}')

Total token count of origin text : 395


일단 전체 크기를 한정한 generator를 구성한다.<br>
향후, 이것은 LSTM을 이용해서 길게 만들 수 있을 것.

_MAX_TOKEN = 512   ' Origin text의 전체 token 개수를 최대 512로 한정 <br>
_MAX_LENGTH = 40   ' generator에 의해 생성하는 문자의 전체 token 개수를 40한정 <br>

이는 512개의 token 원문을 40개의 token 으로 구성된 문장으로 요약 하는것....

In [21]:
# max 512 token으로 만든다. 남는건 padding

_MAX_TOKEN = 512
_MAX_LENGTH = 40
_NOISE_DIM = 100

word_keys = []
word_values = []

for word,index in tokenizer.word_index.items():
    word_keys.append(index)
    word_values.append(word)

current_token_len = len(word_keys)

if current_token_len > _MAX_TOKEN:
    word_keys = word_keys[:_MAX_TOKEN]
    word_values = word_values[:_MAX_TOKEN]
else:
    for i in range(current_token_len+1,_MAX_TOKEN+1):
        word_keys.append(i)
        word_values.append(_MISMATCH_WORD)


In [22]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token')
print('')
for k in word_keys:
  print( f'  {str(k).ljust(8)}    {word_values[k-1]}')
print('---------------------------------------------')


Token table of origin text
---------------------------------------------
 Code         Token

  1           원내대표는
  2           주
  3           것
  4           이라고
  5           며
  6           문재인
  7           않는
  8           문
  9           이라며
  10          이제
  11          풀들이
  12          수
  13          대통령과
  14          공수처는
  15          될
  16          있는
  17          향해
  18          보인다
  19          짓밟힌
  20          국민적
  21          저항에
  22          직면할
  23          정권이
  24          위해
  25          공수처법을
  26          제게
  27          대통령은
  28          '공수처는
  29          없는
  30          야당이
  31          공수처장
  32          사람들이
  33          민주당
  34          그러면서
  35          공수처가
  36          청와대와
  37          왜
  38          그
  39          인정하지
  40          이
  41          공수처를
  42          권력
  43          게
  44          정권의
  45          민주당이
  46          국회에서
  47          그걸
  48          권력은
  49          먼저
  50          주호영
  51          국민의힘

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Concatenate,
                                     Lambda)
import matplotlib.pyplot as plt

In [24]:
# tensor용 Hash table 구성 
keys=tf.constant(word_keys,tf.int32)
values=tf.constant(word_values, tf.string)
# build a lookup table
word_table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys,values),
    default_value=tf.constant(' '),
    name="token_table"
)

noise를 token_table을 통해 text로 변환하는 변환기 구현

In [25]:
# noise 생성
w = tf.random.normal([2,_MAX_LENGTH,_MAX_TOKEN])

In [26]:
import sys

def to_text(w):

    #tf.print(w)
    texts = []
    try:
        for z in w:
            text = ""
            #code = []
            for v in z:
                try:
                    #print(v)
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)

    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return tf.constant(texts,dtype=tf.string)

In [27]:
# to_text 함수의 test
e = to_text(w)
for t in e:
  print(t.numpy().decode('utf-8'))

검찰과 @@@ 앞장 그만 @@@ @@@ 사도들은 22일 보인다 @@@ 고 @@@ 주변을 독립성을 @@@ @@@ 야당의 재정 깜짝할 출범하면 @@@ @@@ 청와대와 @@@ @@@ 수단으로 불법이 '민주당 추미애보다 장외투쟁해 @@@ 여당 개시하면 @@@ 대선 얘기한 상임이사국처럼 밀어넣고 거는 감시하는 
@@@ 자신의 표정으로 @@@ 오만함을 그러면서 '공수처는 국민적 @@@ @@@ 것인데 @@@   것'이라고 막무가내 거는 지도자를 @@@ 한다 추미애보다 @@@ @@@ @@@ @@@ 행사할 될 @@@ 불법·탈법으로 사람들이 마지막으로 개정과 정부를 @@@ 정부를 @@@ 없다'고 무슨 향해 사람들 성토했다 


생성된 text의 embedding 변환기 구현<br>
embedding은 org_text의 embedding과 비교하여 원문과 유사하게 민들기 위한 목적

In [28]:
# 이거 이틀걸림...잘 몰라서 ㅈㄴ 헤맴

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    try:
        for z in w:
            text = ""
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [29]:
# to_embedding 함수의 test
e = to_embedding(w)
for t in e:
  print(t.numpy())

[ 0.13296749  0.02979186 -0.24835426 ... -0.10786135 -0.3754492
  0.35479385]
[ 0.8994536  -0.3465693   0.8225235  ... -0.41565615  0.1934839
 -0.11586138]


In [30]:
# to_compression_ratio

@tf.custom_gradient
def to_compression_ratio(w):
    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))        
        return dy_arr_st
    
    const = 131328 # 512를 순서대로 다 더한값
    value = None
    compression_ratio = []
    try:
        for z in w:
            cr = 0
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    cr += key.numpy()
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    cr += 0                
            compression_ratio.append(cr/const)
        value = tf.constant(compression_ratio,dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad
    

In [31]:
# to_compression_ratio 함수의 test
e = to_compression_ratio(w)
for t in e:
  print(t.numpy())

0.08695785
0.07622898


생성된 text의 morpheme code 변환기 구현<br>
morpheme code는 한국어 문장들(dataset)의 morpheme code와 비교하여 한국어 문법에 가깝게 만들기 위한 목적

In [32]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    texts = []
    codes = []
    value = None
    try:
        for z in w:
            text = ""
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [33]:
# to_morpcoding 함수의 test
e = to_morpcoding(w)
for t in e:
  print(t.numpy())

[ 2  5  1  2 10  1  1  2  9  5 11  6  1  2  1  2  5  2  5  1  1  2  5  2
  2  6  2  6  1  1  2  5  1  1  2  5  2  5  3  2  2  5  2  2  2  1  2  2
  6  1  2  2  5  2  5  6  6  2  5  2  6  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[1 2 5 2 5 1 8 8 3 7 7 2 5 2 9 1 1 2 5 1 2 3 5 2 2 5 2 5 1 6 2 5 1 1 1 1 2
 6 6 1 2 3 2 5 2 9 5 2 5 2 5 2 5 1 2 5 1 8 3 2 2 2 6 2 9 7 2 6 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Network 구성을 위해 사용자 정의 Layer 를 구현

In [34]:
# 이것도 잘 몰라서 하루 걸림... ㅜㅜ

from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float32, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float32 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float32 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])

In [35]:
# 구성한 Layer의 test
e = Post_processing(1024,to_embedding,Tout=tf.float32)(w)
for c in e:
  print(c)

tf.Tensor(
[ 0.13296749  0.02979186 -0.24835426 ... -0.10786135 -0.3754492
  0.35479385], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.8994536  -0.3465693   0.8225235  ... -0.41565615  0.1934839
 -0.11586138], shape=(1024,), dtype=float32)


In [36]:
# 별로 중요하지는 않지만 Lambda layer를 활용하기 위한 assert 함수 구성
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data

In [64]:
# 드디어 generator 구현
# 효과적으로 구성된 것인지는 모르겠음... 이것은 아직 많은 연구가 필요함.
# 또한 LSTM으로 바꾸어 길이의 한게를 극복해야 할 것...

def make_generator_model(max_length,total_words):
    input = Input(shape=(_NOISE_DIM,), dtype='float32') 
    x1 = Dense(1024, use_bias=False)(input)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(1024*2, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)
    
    x1 = Dense(max_length*total_words, use_bias=False, activation='softmax')(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':max_length*total_words})(x1)
    x1 = Reshape((max_length, total_words))(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    txt = Post_processing(0,to_text,Tout=tf.string)(x1)
    emb = Post_processing(1024,to_embedding,Tout=tf.float32)(x1)
    cmr = Post_processing(0,to_compression_ratio,Tout=tf.float32)(x1)
    cod = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    model = Model(input,[txt,emb,cmr,cod])
    
    model.summary()
    return model

generator = make_generator_model(_MAX_LENGTH,_MAX_TOKEN)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 1024)         102400      input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 1024)         4096        dense_7[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_4 (LeakyReLU)       (None, 1024)         0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [38]:
# generator의 test
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,compratios,morpcodes = generator(noise, training=True)
print(texts.shape)
for i,txt in zip(range(len(texts)),texts):
    print(f" {i+1}> {txt.numpy().decode('utf-8')}" )
print(embeddings.shape)
print(compratios.shape)
print(morpcodes.shape)

(3,)
 1> 임명에 바로 신공항 @@@ @@@ 여기에 거짓말이라는 정부를 놓고 불을 강조했다 끔찍한 괴물 @@@ @@@ @@@   이용한다는 밀어붙이고 오만함을 곳간을 아우성치는 @@@ 치는 힘이 @@@ @@@ 사업이니 권력은 @@@ 불면 법위에 @@@ 감시하는 앉힐 민초의 시행도 공직자들을 @@@ 고 
 2> 예측했다 @@@ @@@ 180석의 @@@ 공수처는 @@@ 한 일 공수처가 @@@ 온통 @@@ 행태가 사건이 앞장 공수처장이 @@@ @@@ 봐야 @@@ 윤석열 @@@ 막무가내 '바람보다 처리장이 보장하는 완성된다 @@@ @@@ 먼저 일어나는' @@@ @@@ @@@ 공수처가 다 저항에 힘을 것이라고 
 3> 사업이니 거리낌 부정하는 @@@ 원내대표인 곳간을 온통 바람 大亂大治 그러면서 집권'의 윽박지른다 누구나 180석의 @@@ 끔찍한 꼬집었다 한다 주호영 원내대표인 독립성을 거짓말이라는 청보리 건설 @@@ 정죄하는 얻기 목숨바쳐 얻기 봉착하자 야당이 대통령이 정의당을 일을 @@@ 강조했다 곳간을 통치기술 거짓말이라는 통치기술 
(3, 1024)
(3,)
(3, 128)


# Discriminator 구현

먼저 요약을 구분하기 위한 discriminator_summ 구현

In [39]:
# 문장에 대한 embeddings를 이용하여 org_text_emb (org_text의 embedding)과의 유사도를 계산한다.
import scipy

@tf.custom_gradient
def to_similarity(w):

    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],1024),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],1024))
        return dy_arr_st

    similarities = []
    value = None
    try:
        for embedding in w:
            distances = scipy.spatial.distance.cdist([embedding], [org_text_emb], "cosine")[0]
            similarities.append(distances[0])
            
        value = tf.constant(similarities,dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad


In [65]:
def make_discriminator_model():
    input_emb = Input(shape=(1024,), dtype='float32') 
    x1 = Post_processing(0,to_similarity,Tout=tf.float32)(input_emb)
    x1 = Reshape((1,))(x1)    
    input_cmp = Input(shape=(), dtype='float32') 
    imput_mrp = Input(shape=(_MAX_MORP_LENGTH,), dtype='int32')

    x2 = Reshape((1,))(input_cmp)
    x3 = Dense(256)(imput_mrp)
    x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    concatted = Concatenate(axis=1)([x1, x2, x3])
    x4 = Flatten()(concatted)
    x4 = Dense(64, use_bias=False)(x4)
    x4 = BatchNormalization()(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dense(1)(x4)
    
    model = Model([input_emb,input_cmp,imput_mrp],x4)
    
    model.summary()
    
    return model

discriminator = make_discriminator_model()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1024)]       0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 256)          33024       input_10[0][0]                   
__________________________________________________________________________________________________
post_processing_11 (Post_proces (None,)              0           input_8[0][0]                    
____________________________________________________________________________________________

In [41]:
# discriminator test

predict = discriminator([embeddings,compratios])
print(predict)

tf.Tensor(
[[-0.12610126]
 [-0.14593472]
 [-0.1451394 ]], shape=(3, 1), dtype=float32)


# GAN 을 이용한 loss 의 gradient 구현 --> 빡심

In [42]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [66]:
# 디짐
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        # real에 가까우려면 discriminator의 학습은 real_embedding 이 zero (0)에 가깝게 학습시켜야 함.
        # 하지만 압축율의 개념으로는 본래는 ones (1)가 맞음.
        real_output = discriminator([real_embedding,np.zeros(len(real_embedding)),real_morpcoding], training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [67]:
EPOCHS = 2
noise_dim = _NOISE_DIM
# 요약문 생성의 확인을 위해 10개의 문장을 생성하고 train과정에서 각 epoch마다 변화를 확인한다.
seed = tf.random.normal([10, noise_dim])

In [68]:
# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []
texts,embeddings,compratios,morpcodes = generator(seed,training=False)
#count = 0
for t in texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,org_text_emb)))

Similarity score: 6.154460643045245


In [46]:
# Print iterations progress
class ProgressBar:
    # pb = ProgressBar(total=20, prefix = 'Epoch 1')
    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
    # pb.printProgress(1,'~~~~')
    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '-' * (self.length - filledLength)
        print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
        # Print New Line on Complete
        if self.ite == self.total: 
            print()

In [69]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
    # A. For each epoch, do the following:
    for epoch in range(epochs):
        start = time.time()
        pb = ProgressBar(total=BATCH_COUNT, prefix = f'Epoch:{str(epoch+1)}/{epochs}')
        pb.printProgress(0,'Start batch.')
        # 1 - For each batch of the epoch, 
        for batch_num,(emb_batch_set,cod_batch_set) in zip(range(len(dataset)),dataset):
            # 1.a - run the custom "train_step" function
            # we just declared above
            #print(image_batch.shape)
            train_step(emb_batch_set,cod_batch_set)
            pb.printProgress(+1,f'Time for batch {batch_num + 1}/{BATCH_COUNT} is {time.time()-start} sec')
        # 4 - Print out the completed epoch no. and the time spent
        #print (f'Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        count = 0
        queries = []
        for i,t in  zip(range(len(texts)),texts):
            summary_text = t.numpy().decode('utf-8')
            print(f'{i+1} > {summary_text}')
            queries.append(summary_text)
            c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
            count += len(c)
        print(f'Mismatch count:{count} Similarity score:{str(similarity_score(queries,org_text_emb))}')
        print('')

In [ ]:
train(dataset, EPOCHS)

Epoch:1/2 |--------------------| 1.0%   Time for batch 1/100 is 13.428307294845581 sec